In [17]:
import os, glob, pathlib
import igibson
import json

In [2]:
obj_metadata_files = list(glob.glob(os.path.join(igibson.ig_dataset_path, "objects", "*", "*", "misc", "metadata.json")))

In [3]:
len(obj_metadata_files)

1216

In [4]:
obj_metadatas = []
for x in obj_metadata_files:
    with open(x, "r") as f:
        obj_metadatas.append(json.load(f))

In [5]:
len(obj_metadatas)

1216

In [6]:
# Take a look at the unique base-level keys
{key for om in obj_metadatas for key in om.keys()}

{'base_link_offset',
 'bbox_size',
 'link_bounding_boxes',
 'links',
 'object_parts',
 'openable_both_sides',
 'openable_joint_ids',
 'orientations'}

In [15]:
import yaml
IN_DATASET_ROOT = r"C:\Users\Cem\research\iGibson-dev\igibson\data\ig_dataset"
TRANSLATION_PATH = os.path.join(IN_DATASET_ROOT, "metadata", "model_rename.yaml")
with open(TRANSLATION_PATH, "r") as f:
    TRANSLATION_DICT = yaml.load(f, Loader=yaml.SafeLoader)

In [27]:
def find_sliceable():
    old_interesting = [(pathlib.Path(f).parts[-4], pathlib.Path(f).parts[-3]) for f, x in zip(obj_metadata_files, obj_metadatas) if "object_parts" in x]
    old_names = [TRANSLATION_DICT[old_category_name + "/" + old_model_name].split("/") for old_category_name, old_model_name in old_interesting]
    print("dvc add", " ".join([f"cad/objects/legacy_{cat}-{mod}/processed.max cad/objects/legacy_{cat}-{mod}/textures" for cat, mod in old_names]))
    
find_sliceable()

dvc add cad/objects/legacy_apple-agveuv/processed.max cad/objects/legacy_apple-agveuv/textures cad/objects/legacy_apple-omzprq/processed.max cad/objects/legacy_apple-omzprq/textures cad/objects/legacy_apricot-qmwmwm/processed.max cad/objects/legacy_apricot-qmwmwm/textures cad/objects/legacy_chestnut-fmomat/processed.max cad/objects/legacy_chestnut-fmomat/textures cad/objects/legacy_coconut-qwriih/processed.max cad/objects/legacy_coconut-qwriih/textures cad/objects/legacy_kiwi-khupqq/processed.max cad/objects/legacy_kiwi-khupqq/textures cad/objects/legacy_lemon-vuwehu/processed.max cad/objects/legacy_lemon-vuwehu/textures cad/objects/legacy_mushroom-zlukhz/processed.max cad/objects/legacy_mushroom-zlukhz/textures cad/objects/legacy_orange-ucstpm/processed.max cad/objects/legacy_orange-ucstpm/textures cad/objects/legacy_peach-wrutgk/processed.max cad/objects/legacy_peach-wrutgk/textures cad/objects/legacy_pear-ngtwvy/processed.max cad/objects/legacy_pear-ngtwvy/textures cad/objects/legac

In [8]:
interesting[0]

('C:\\Users\\Cem\\research\\iGibson-dev\\igibson\\data/ig_dataset\\objects\\apple\\00_0\\misc\\metadata.json',
 {'base_link_offset': [0.0002865009009838104,
   -0.00014150142669677734,
   0.0011289995163679123],
  'bbox_size': [0.06453900039196014, 0.06561899930238724, 0.07745200023055077],
  'orientations': [],
  'object_parts': [{'category': 'apple',
    'model': '00_1',
    'pos': [0.005, 0.0, 0.0],
    'orn': [0.0, 0.0, 0.0, 1.0]},
   {'category': 'apple',
    'model': '00_1',
    'pos': [-0.005, 0.0, 0.0],
    'orn': [0.0, 0.0, 1.0, 0.0]}],
  'link_bounding_boxes': {'base_link': {'collision': {'axis_aligned': {'extent': [0.064539,
       0.06561900000000001,
       0.077452],
      'transform': [[1.0, 0.0, 0.0, 0.00028650000000000203],
       [0.0, 1.0, 0.0, -0.00014149999999999927],
       [0.0, 0.0, 1.0, 0.0011290000000000015],
       [0.0, 0.0, 0.0, 1.0]]},
     'oriented': {'extent': [0.06414657394238396,
       0.06600148273623291,
       0.06830139396790418],
      'transfor

In [34]:
link_ids = set(key for _, x in interesting for key in x["links"].keys())
print(link_ids)

{'water_source', 'heat_source', 'cleaning_tool_area', 'slicer', 'toggle_button'}


In [36]:
for link_id in link_ids:
    id_interesting = [(f, x) for f, x in zip(obj_metadata_files, obj_metadatas) if "links" in x and link_id in x["links"]]
    link_features = set(key for _, x in id_interesting for key in x["links"][link_id].keys())
    print(link_id, ":", link_features)

water_source : {'xyz', 'size', 'rpy', 'geometry'}
heat_source : {'xyz', 'size', 'rpy', 'geometry'}
cleaning_tool_area : {'xyz', 'size', 'rpy', 'geometry'}
slicer : {'xyz', 'size', 'rpy', 'geometry'}
toggle_button : {'xyz', 'size', 'rpy', 'geometry'}


In [45]:
# Which ones have non-none Geometry?
geo_interesting = [(f, x) for f, x in zip(obj_metadata_files, obj_metadatas) if "links" in x and any(linkdata["geometry"] is not None for linkdata in x["links"].values())]
geo_interesting_link_ids = set(key for _, x in geo_interesting for key in x["links"].keys())
geo_types = set(link["geometry"] for _, x in geo_interesting for link in x["links"].values())

print(geo_interesting_link_ids)
print(geo_types)
print(geo_interesting[0][1]["links"])

{'slicer', 'cleaning_tool_area'}
{'box'}
{'cleaning_tool_area': {'geometry': 'box', 'size': [15.547669797390698, 41.33866354227065, 8.505709015205484], 'xyz': [-9.580486297607422, 0.66823810338974, 4.875622749328613], 'rpy': [2.7894475381535686e-09, 0.26179938375919576, -0.26179935794962716]}}
